## Turkish News

### Dataset Preprocessing

In [ ]:
# !pip install datasets

In [ ]:
import os
import pandas as pd
import numpy as np
from datasets import load_dataset

from src.utils import get_project_config
from src.utils import missing_values
import time
from datetime import datetime
import warnings

# import matplotlib.pyplot as plt
# import seaborn

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [ ]:
# Set Constants
ROOT_PATH = os.getcwd()
CFG_PATH = os.path.join(ROOT_PATH, 'cfg')
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
EMBEDDING_DATA_PATH = os.path.join(DATA_PATH, 'embedding')

os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(EMBEDDING_DATA_PATH, exist_ok=True)

### Import Dataset: TR News Datasets

In [ ]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
df_tr_news_train_data = pd.read_parquet("hf://datasets/denizzhansahin/Turkish_News-2024/" + splits["train"])
df_tr_news_val_data = pd.read_parquet("hf://datasets/denizzhansahin/Turkish_News-2024/" + splits["validation"])
df_tr_news_data = pd.concat([df_tr_news_train_data, df_tr_news_val_data], axis=0)

df_tr_news_data = df_tr_news_data[['Baslik', 'Ozet', 'Icerik', 'Kategori']]
df_tr_news_data.to_csv(os.path.join(RAW_DATA_PATH, "tr_news_data.csv.gz"), index=False, compression='gzip')
df_tr_news_data.head()

Baslik  \
0                                          Aksaray’dan Buckingham Sarayı’na   
1                Malatya'da trafik kazasında 1 çocuk öldü, 2 kişi yaralandı   
2                              "Kozmik oda kumpas" davasında cezalara onama   
3                      Antalya Doğu Garajı'ndaki nekropol alanı müze olacak   
4  SBÜ'de Jandarma adına eğitim görecek adayların sınav sonuçları açıklandı   

                                                                                                                                                                                                              Ozet  \
0  Buckingham Sarayı’ndan, Atatürk’ün özel koleksiyonuna kadar kıymetli birçok halı Aksaray’ın Sultanhanı ilçesinde tamir ediliyor. Ünü ülke sınırlarını aşan ilçe, halı tamiri işinde dünyanın en gözde adresi...   
1                                                                                                                       Malatya'da trambüse çarpan hafif ticari araçtaki 3 yaşındaki çocuk öldü, 2 kişi yaralandı.   
2                   Yargıtay Ceza Genel Kurulu, Ankara Seferberlik Bölge Başkanlığının "kozmik odaları"nda günlerce süren aramalarla devam eden soruşturmada görev alan hakim ve savcılara verilen cezaları onadı.   
3                                                          Antalya Doğu Garajı Kültür Merkezi inşaatı için başlatılan kazı çalışmalarında ortaya çıkan 2 bin 500 yıllık nekropol alanı müze olarak hizmet verecek.   
4                                                                          Sağlık Bilimleri Üniversitesinde (SBÜ), Jandarma Genel Komutanlığı nam ve hesabına öğrenim görecek adayların sınav sonuçları açıklandı.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

### Show Metadata, Shape, Statistics and Missing Table

In [ ]:
df_tr_news_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27387 entries, 0 to 8216
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Baslik    27387 non-null  object
 1   Ozet      27387 non-null  object
 2   Icerik    27387 non-null  object
 3   Kategori  27387 non-null  object
dtypes: object(4)
memory usage: 1.0+ MB


In [ ]:
df_tr_news_data.shape

(27387, 4)

In [ ]:
df_tr_news_data['Kategori'].nunique()

14

In [ ]:
df_tr_news_data['Kategori'].value_counts(normalize=False)

,count
Kategori,
Dünya,8390
Gündem,5480
Türkiye,3870
Spor,2000
Eğitim,1780
Kültür-Sanat,1710
Yaşam,1280
Savunma,952
Ekonomi,750


In [ ]:
missing_values(df=df_tr_news_data, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
Baslik,0,0.0
Ozet,0,0.0
Icerik,0,0.0
Kategori,0,0.0


In [ ]:
# df_tr_news_data.head(5)

### Sampling Data

In [ ]:
# Neden Yapıyoruz
# 1- Balanced Dataset
# 2- Computing Power

In [ ]:
news_cats = ['Dünya', 'Gündem', 'Türkiye', 'Spor', 'Eğitim'] 
# Data Cover: %78,5

df_tr_news_data_balanced = pd.DataFrame()
for cat in news_cats:
    df_tr_news_data_cat = df_tr_news_data[df_tr_news_data['Kategori'] == cat].sample(n=1500, random_state=34)
    df_tr_news_data_balanced = pd.concat([df_tr_news_data_balanced, df_tr_news_data_cat])

df_tr_news_data = df_tr_news_data_balanced.sample(frac=1, random_state=34).reset_index(drop=True)

print(df_tr_news_data['Kategori'].value_counts(normalize=False))

Kategori
Türkiye    1500
Dünya      1500
Spor       1500
Gündem     1500
Eğitim     1500
Name: count, dtype: int64


In [ ]:
df_tr_news_data.head(5)

Baslik  \
0                                                  İstanbul'da çakar ve emniyet şeridi denetimi yapıldı   
1                   İsrail basını: ABD ile Gazze'de çok uluslu güç oluşturulması için ilerleme sağlandı   
2                                Josep Borrell'den İsrail'e tepki: Nereye tahliye edeceksiniz, Ay'a mı?   
3  İskoçya Başbakanı'nın eşi, Gazze'deki aile üyelerinin Türkiye'nin yardımıyla kurtarıldığını açıkladı   
4                                               Kadroda düşünülmedi, formayı kaptı, rekor bedelle gitti   

                                                                                                                                                                                                                                     Ozet  \
0                                                                                                          İstanbul'da polis ekipleri, usulsüz çakar-siren tertibatı kullanımı ve emniyet şeridi ihlaline yönelik denetim gerçekleştirdi.   
1                                                                                   İsrail Savunma Bakanı Yoav Gallant'ın, Gazze'de çok uluslu güç oluşturulması konusunda ABD'de yaptığı görüşmelerde ilerleme kaydedildiği öne sürüldü.   
2  Avrupa Birliği (AB) Dış İlişkiler ve Güvenlik Politikası Yüksek Temsilcisi Josep Borrell, İsrail'in Gazze'ye yönelik saldırılarında yaşanan sivil kayıplara tepki gösteren ülkelere, İsrail'e silah ihracatını durdurma çağrısı yaptı.   
3                         İskoçya Bölgesel Başbakanı Hamza Yusuf'un eşi Nadia El-Nakla, Gazze'deki kardeşinin eşi ve çocuklarının Türkiye tarafından bölgeden çıkarıldığını ve geçici koruma statüsüyle Türkiye'ye getirildiğini söyledi.   
4                                                     Galatasaray'dan Almanya'nın dünyaca ünlü kulübü Bayern Münih'e giden Fransız futbolcu Sacha Boey'un 2,5 sezonluk Türkiye kariyeri, filmlere konu olabilecek gelişmelere sahne oldu.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
df_tr_news_data.shape

(7500, 4)

In [ ]:
df_tr_news_data["OzetLength"] = df_tr_news_data["Ozet"].apply(len)
df_tr_news_data["OzetWordCount"] = df_tr_news_data['Ozet'].str.split().str.len()

In [ ]:
df_tr_news_data[["OzetLength", "OzetWordCount"]].describe(percentiles=[0.80,0.85,0.90,0.95,0.99]).T

,count,mean,std,min,50%,80%,85%,90%,95%,99%,max
OzetWordCount,7500.0,21.491467,8.134327,5.0,20.0,28.0,30.0,33.0,37.0,44.0,51.0


In [ ]:
df_tr_news_data[df_tr_news_data['OzetWordCount']==5]

,Baslik,Ozet,Icerik,Kategori,OzetLength,OzetWordCount
6567,Doğu Akdeniz için sağanak uyarısı,Doğu Akdeniz'de sağanak uyarısında bulunuldu.,"Meteoroloji Genel Müdürlüğünden yapılan açıklamaya göre, Doğu Akdeniz'de yağışların, bu akşamdan itibaren yarın sabaha kadar Osmaniye ve Hatay çevreleri ile Adana'nın doğu ve Kahramanmaraş'ın güney kesimlerinde yerel kuvvetli gök gürültülü sağanak şeklinde düşmesi bekleniyor. Vatandaşların, ani sel, su baskını, yıldırım, ulaşımda aksama gibi olumsuzluklara karşı dikkatli olması gerekiyor.",Türkiye,45,5


In [ ]:
df_tr_news_data.groupby(by=['Kategori'], as_index=False).agg(OzetWordCount_Mean=('OzetWordCount','mean'),
                                                                  OzetLength_Mean=('OzetLength','mean'))

,Kategori,OzetWordCount_Mean,OzetLength_Mean
0,Dünya,21.986667,179.368000
1,Eğitim,23.092000,183.160667
2,Gündem,25.961333,205.670667
3,Spor,18.909333,147.216000
4,Türkiye,17.508000,138.000667


### Embedding

In [ ]:
df = df_tr_news_data[['Ozet', 'Kategori']]
df[0:10]

,Ozet,Kategori
0,"İstanbul'da polis ekipleri, usulsüz çakar-siren tertibatı kullanımı ve emniyet şeridi ihlaline yönelik denetim gerçekleştirdi.",Türkiye
1,"İsrail Savunma Bakanı Yoav Gallant'ın, Gazze'de çok uluslu güç oluşturulması konusunda ABD'de yaptığı görüşmelerde ilerleme kaydedildiği öne sürüldü.",Dünya
2,"Avrupa Birliği (AB) Dış İlişkiler ve Güvenlik Politikası Yüksek Temsilcisi Josep Borrell, İsrail'in Gazze'ye yönelik saldırılarında yaşanan sivil kayıplara tepki gösteren ülkelere, İsrail'e silah ihracatını durdurma çağrısı yaptı.",Dünya
3,"İskoçya Bölgesel Başbakanı Hamza Yusuf'un eşi Nadia El-Nakla, Gazze'deki kardeşinin eşi ve çocuklarının Türkiye tarafından bölgeden çıkarıldığını ve geçici koruma statüsüyle Türkiye'ye getirildiğini söyledi.",Dünya
4,"Galatasaray'dan Almanya'nın dünyaca ünlü kulübü Bayern Münih'e giden Fransız futbolcu Sacha Boey'un 2,5 sezonluk Türkiye kariyeri, filmlere konu olabilecek gelişmelere sahne oldu.",Spor
5,"İsrail Genelkurmay Başkanı Herzi Halevi, abluka altındaki Gazze Şeridi’ne yönelik saldırıların devamı için gerekli planları onayladıklarını açıkladı.",Dünya
6,"Beşiktaş Kulübü, 2023-2024 futbol sezonu ikinci transfer ve tescil döneminde kadrosuna kattığı futbolcular için imza töreni düzenledi.",Spor
7,Yarın yapılacak Mahalli İdareler Genel Seçimleri'ne ilişkin siyasi partilerin propaganda serbestliği saat 18.00'de sona erdi.,Gündem
8,"Ukrayna Devlet Başkanı Volodimir Zelenskiy, Kırım Tatar Türk halkının lideri Mustafa Abdülcemil Kırımoğlu'na ""Ukrayna Kahramanı"" unvanı ile devlet nişanı takdim etti.",Dünya
9,"İspanya Meclisi, ülkede büyük siyasi tartışmalara neden olan, ayrılıkçı Katalanlara af getirilmesini öngören yasa tasarısının, yasal süreçle ilerleyip Meclis'te görüşülmesine onay verdi.",Dünya


In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model = [
    'sentence-transformers/all-MiniLM-L12-v2',
    "jinaai/jina-embeddings-v3",
    "intfloat/multilingual-e5-large-instruct",
    "BAAI/bge-m3",
    "thenlper/gte-large"
]

In [ ]:
for model_name in embedding_model:
    try:
        embedding_duration = datetime.now()
        print(f"Embedding Model Name: {model_name} - {datetime.now()} ")
        model = SentenceTransformer(model_name, trust_remote_code=True)
        model_output_file = os.path.join(
            EMBEDDING_DATA_PATH, f"news_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        embeddings = model.encode(df['Ozet'].tolist())
        embedding_dim = len(embeddings[0])
        print(f"Embedding Dimension: {embedding_dim}")
        embedding_columns = [f'embedding_{i}' for i in range(embedding_dim)]
        df_embedding = pd.DataFrame(embeddings, columns=embedding_columns)
        df_data = pd.concat([df, df_embedding], axis=1)
        df_data.to_csv(model_output_file, compression='gzip', index=False)
        print(f"Embedding Duration: {datetime.now() - embedding_duration}")
        print("+ " * 15)
    except Exception as exc:
        print("Exception: ", exc)
        print("x " * 15)

Embedding Model Name: sentence-transformers/all-MiniLM-L12-v2 - 2024-12-02 18:15:42.158520 
Embedding Dimension: 384
Embedding Duration: 0:00:28.519620
+ + + + + + + + + + + + + + + 
Embedding Model Name: jinaai/jina-embeddings-v3 - 2024-12-02 18:16:10.678310 


Embedding Dimension: 1024
Embedding Duration: 0:02:23.349771
+ + + + + + + + + + + + + + + 
Embedding Model Name: intfloat/multilingual-e5-large-instruct - 2024-12-02 18:18:34.028200 
Embedding Dimension: 1024
Embedding Duration: 0:02:09.299802
+ + + + + + + + + + + + + + + 
Embedding Model Name: BAAI/bge-m3 - 2024-12-02 18:20:43.328881 
Embedding Dimension: 1024
Embedding Duration: 0:02:19.444222
+ + + + + + + + + + + + + + + 
Embedding Model Name: thenlper/gte-large - 2024-12-02 18:23:02.774432 
Embedding Dimension: 1024
Embedding Duration: 0:03:16.547137
+ + + + + + + + + + + + + + + 


In [ ]:
for f in os.listdir(EMBEDDING_DATA_PATH):
    # print(f)
    if f.split("_")[0] == "news":
        df_emb_data = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, f), compression='gzip')
        print(f"Emb. File Name:{f} & Shape: {df_emb_data.shape}")
        print(df_emb_data.head(0))
        print("+ " * 25)

Emb. File Name:news_BAAI__bge-m3_embeddings.csv.gz & Shape: (7500, 1026)
Empty DataFrame
Columns: [Ozet, Kategori, embedding_0, embedding_1, embedding_2, embedding_3, embedding_4, embedding_5, embedding_6, embedding_7, embedding_8, embedding_9, embedding_10, embedding_11, embedding_12, embedding_13, embedding_14, embedding_15, embedding_16, embedding_17, embedding_18, embedding_19, embedding_20, embedding_21, embedding_22, embedding_23, embedding_24, embedding_25, embedding_26, embedding_27, embedding_28, embedding_29, embedding_30, embedding_31, embedding_32, embedding_33, embedding_34, embedding_35, embedding_36, embedding_37, embedding_38, embedding_39, embedding_40, embedding_41, embedding_42, embedding_43, embedding_44, embedding_45, embedding_46, embedding_47, embedding_48, embedding_49, embedding_50, embedding_51, embedding_52, embedding_53, embedding_54, embedding_55, embedding_56, embedding_57, embedding_58, embedding_59, embedding_60, embedding_61, embedding_62, embedding_63,